In [1]:
from tifffile import imread
from functions import *
import json
import geojson
import os

In [2]:
WSI_path = r'\\10.99.68.54\Digital pathology image lib\HubMap Skin TMC project\CellViT\inference_pilot'
model_path = r'\\10.162.80.16\Andre_expansion\data\Stardist\PDAC model\models\lea_model'

In [3]:
# WSIs = [os.path.join(WSI_path, f) for f in os.listdir(WSI_path) if f.endswith('.ndpi')]  # change if using tif or czi file
WSIs = [os.path.join(WSI_path, f) for f in os.listdir(WSI_path) if f.endswith('.svs')]  # change if using tif or czi file
model = load_model(model_path)

base_model.py (149): output path for model already exists, files may be overwritten: \\10.162.80.16\Andre_expansion\data\Stardist\PDAC model\models\lea_model\offshoot_model


Using default values: prob_thresh=0.5, nms_thresh=0.4.
Overriding defaults: Thresholds(prob=0.4872387821889486, nms=0.3) 


In [5]:
model = load_model(model_path)
date = '2_23_24'

out_pth = os.path.join(WSI_path, f'StarDist_{date}')
if not os.path.exists(out_pth):
    os.mkdir(out_pth)

out_pth_json = os.path.join(out_pth, 'json')
out_pth_tif = os.path.join(out_pth, 'tif')
print(out_pth_json)

if not os.path.exists(out_pth_json):
    os.mkdir(out_pth_json)

if not os.path.exists(out_pth_json):
    os.mkdir(out_pth_json)

if not os.path.exists(out_pth_tif):
    os.mkdir(out_pth_tif)

Using default values: prob_thresh=0.5, nms_thresh=0.4.
Overriding defaults: Thresholds(prob=0.4872387821889486, nms=0.3) 

\\10.99.68.54\Digital pathology image lib\HubMap Skin TMC project\CellViT\inference_pilot\StarDist_2_23_24\json


In [6]:
# Segment all WSIs -- takes about 2-5 minutes per whole slide image to segment, about 3 minutes to save geojson file
for img_pth in WSIs:
    try:
        name = os.path.basename(img_pth)

        if not os.path.exists(os.path.join(out_pth_json, (name[:-5] + '.json'))):
            print(f'Starting {name}')

            img = imread(img_pth)
            img = img/255  # normalization used to train model
            _, polys = model.predict_instances_big(img, axes='YXC', block_size=4096, min_overlap=128, context=128, n_tiles=(4,4,1))

            print('Saving json...')
            save_json_from_WSI_pred(polys, out_pth_json, name)

            # tif file is like 3 GB usually, so only uncomment next part if you are ok with that
            #print('Saving tif...')
            #imwrite(os.path.join(out_pth_tif, name[:-5] + '.tif'), labels)
        else:
            print(f'Skipping {name}')
    except:
        print(f'skipping {img_pth}, probably bc its too big...')

Starting TCGA-V5-A7RE-11A-01-TS1.57401526-EF9E-49AC-8FF6-B4F9652311CE.svs
effective: block_size=(4096, 4096, 3), min_overlap=(128, 128, 0), context=(128, 128, 0)


100%|██████████| 12/12 [00:26<00:00,  2.22s/it]


Saving json...
Finished TCGA-V5-A7RE-11A-01-TS1.57401526-EF9E-49AC-8FF6-B4F9652311C.json


In [7]:
json_pth_list = sorted([os.path.join(out_pth_json,file) for file in os.listdir(out_pth_json) if file.endswith(".json")])
print(json_pth_list)

['\\\\10.99.68.54\\Digital pathology image lib\\HubMap Skin TMC project\\CellViT\\inference_pilot\\StarDist_2_23_24\\json\\TCGA-V5-A7RE-11A-01-TS1.57401526-EF9E-49AC-8FF6-B4F9652311C.json']


make geojson file to load over ndpi file in qupath

In [10]:
for p, file in enumerate([json_pth_list[0]]):
    nm = file.split('\\')[-1]
    new_fn = os.path.join(out_pth, nm[:-5] + '.geojson')
    print(f'{p} / {len(json_pth_list)}')
    print(nm)
    
    if not os.path.exists(new_fn):
    
        segmentation_data = json.load(open(file))
        
        data_list = format_seg_data(segmentation_data, ds_amt)
    
        GEOdata = []
        
        for j, (centroid, contour) in enumerate(data_list):
            
            #if j == 100000:
            #    break
            
            centroid = [centroid[0] + 0, centroid[1] + 0]
            # xy coordinates are swapped, so I reverse them here with xy[::-1]
            # note: add 1 to coords to fix 0 indexing vs 1 index offset
            contour = [[coord+0 for coord in xy[::-1]] for xy in contour]  # Convert coordinates to integers
            contour.append(contour[0]) # stardist doesn't close the circle, needed for qupath
        
            # Create a new dictionary for each contour
            dict_data = {
                "type": "Feature",
                "id": "PathCellObject",
                "geometry": {
                    "type": "Polygon",
                    "coordinates": [contour]
                },
                "properties": {
                    'objectType': 'annotation',
                    'classification': {'name': 'Nuclei', 'color': [97, 214, 59]}
                }
            }
        
            GEOdata.append(dict_data)
        
        with open(new_fn,'w') as outfile:
            geojson.dump(GEOdata,outfile)
        print('Finished',new_fn)
    
    else:
        print(f'skipping {new_fn}')

0 / 1
TCGA-V5-A7RE-11A-01-TS1.57401526-EF9E-49AC-8FF6-B4F9652311C.json
Finished \\10.99.68.54\Digital pathology image lib\HubMap Skin TMC project\CellViT\inference_pilot\StarDist_2_23_24\TCGA-V5-A7RE-11A-01-TS1.57401526-EF9E-49AC-8FF6-B4F9652311C.geojson


Make pickle file with nuclear features output

In [11]:
from analysis_functions import *

In [22]:
pkl_pth = os.path.join(out_pth_json,'nuclei_features_pkls')
if not os.path.exists(pkl_pth): os.mkdir(pkl_pth)
print(pkl_pth)

\\10.99.68.54\Digital pathology image lib\HubMap Skin TMC project\CellViT\inference_pilot\StarDist_2_23_24\json\nuclei_features_pkls


In [23]:
for i, json_f_name in enumerate(json_pth_list):
    
    nm = json_f_name.split('\\')[-1].split('.')[0]
    
    outnm = os.path.join(pkl_pth, f'{nm}.pkl')
    print(outnm)
    
    if not os.path.exists(outnm):
        
        HE_20x_WSI = imread(WSIs[i])
        
        print(WSIs[i])
        print(json_f_name)
        
        try:
            segmentation_data = json.load(open(json_f_name))
        except:
            print(f'error reading json... Skipping {json_f_name}')
            continue
    
        centroids = [nuc['centroid'][0] for nuc in segmentation_data]
        contours = [nuc['contour'] for nuc in segmentation_data]
        contours_fixed = fix_contours(contours)
        
        # part of code below gets rgb intensity values within each nucleus contour, to do this efficiently, it crops
        # a small part of the image for each nucleus. The offset variable determines how big the crop is. 30 should 
        # be fine for 20x images with normal/large sized nuclei but you can play with it yourself if you want
        # it will skip nuclei that are by the edge and thus can't crop that image, it calls these intensities -1
        
        offset = 30  # radius of image that gets cropped from WSI, used for getting rgb intensity average inside nuc contour
        
        centroids_np = np.array(centroids)  # for other formatting
        contours_np = np.array(contours)
        
        r_avg_list = []
        g_avg_list = []
        b_avg_list = []
        
        areas = []
        perimeters = []
        circularities = []
        aspect_ratios = []
        image_ids = []
        classes = []
        
        compactness_a, eccentricity_a, euler_number_a, extent_a, form_factor_a, maximum_radius_a, mean_radius_a, median_radius_a, minor_axis_length_a, major_axis_length_a, orientation_degrees_a = [], [], [], [], [], [], [], [], [], [], []
        
        np_centroids = np.array(centroids)
        
        for j in range(len(contours_fixed)):
            #break
            
            centroid = centroids[j]
            # print(f'centroid: {centroid}')
            contour_raw = copy.copy(contours_fixed[j])  # used for rgb intensities
             
            # get rbg intensity averages
            r_avg, g_avg, b_avg = get_rbg_avg(centroid, contour_raw, offset, HE_20x_WSI)
            # print(r_avg, g_avg, b_avg)
            
            r_avg_list.append(r_avg)
            g_avg_list.append(g_avg)
            b_avg_list.append(b_avg)
            
            contour = contours_np[j][0].transpose()  # used for other stuff, too lazy to make formatting the same
            area = cntarea(contour)
            perimeter = cntperi(contour)
            circularity = 4 * np.pi * area / perimeter ** 2
            MA = cntMA(contour)
            [MA, ma, orientation] = MA
            aspect_ratio = MA / ma
            #center_x = centroid[0]
            #center_y = centroid[1]
            
            cent_x = np_centroids[j,0]
            cent_y = np_centroids[j,1]
            
            #compactness and form_factor are stupid because they are basically same as circularity, maybe extent too
            
            compactness = perimeter ** 2 / area
            eccentricity = np.sqrt(1 - (ma / MA) ** 2)
            extent = area / (MA * ma)
            form_factor = (perimeter ** 2) / (4 * np.pi * area)
            major_axis_length = MA
            maximum_radius = np.max(np.linalg.norm(contour - centroid, axis=1))
            mean_radius = np.mean(np.linalg.norm(contour - centroid, axis=1))
            median_radius = np.median(np.linalg.norm(contour - centroid, axis=1))
            minor_axis_length = ma
            orientation_degrees = np.degrees(orientation)
            
            areas.append(area)
            perimeters.append(perimeter)
            circularities.append(circularity)
            aspect_ratios.append(aspect_ratio)
    
            # additional features
            compactness_a.append(compactness)
            eccentricity_a.append(eccentricity)
            extent_a.append(extent)
            form_factor_a.append(form_factor)
            maximum_radius_a.append(maximum_radius)
            mean_radius_a.append(mean_radius)
            median_radius_a.append(median_radius)
            minor_axis_length_a.append(minor_axis_length)
            major_axis_length_a.append(major_axis_length)
            orientation_degrees_a.append(orientation_degrees)
            
            
        # exit loop
            
        dat = {
            'Centroid_x': np_centroids[:,1],
            'Centroid_y': np_centroids[:,0],
            'Area': areas,
            'Perimeter': perimeters,
            'Circularity': circularities,
            'Aspect Ratio': aspect_ratios,
            'compactness' : compactness_a,
            'eccentricity' : eccentricity_a,
            'extent' : extent_a,
            'form_factor' : form_factor_a,
            'maximum_radius' : maximum_radius_a,
            'mean_radius' : mean_radius_a,
            'median_radius' : median_radius_a,
            'minor_axis_length' : minor_axis_length_a,
            'major_axis_length' : major_axis_length_a,
            'orientation_degrees' : orientation_degrees_a,
            'r_mean_intensity' : r_avg_list,
            'g_mean_intensity' : g_avg_list,
            'b_mean_intensity' : b_avg_list,
            #'slide_num': nm[-4:]  # fix this for your own needs, this gets slide number for my monkey fetus
        }
    
        df = pd.DataFrame(dat).astype(np.float32)  # save a little space with float16 type -> Edit 2 months later, this did not save time.
        
        df.to_pickle(outnm)
        #break
    else:
        print('skipping')

\\10.99.68.54\Digital pathology image lib\HubMap Skin TMC project\CellViT\inference_pilot\StarDist_2_23_24\json\nuclei_features_pkls\TCGA-V5-A7RE-11A-01-TS1.pkl
\\10.99.68.54\Digital pathology image lib\HubMap Skin TMC project\CellViT\inference_pilot\TCGA-V5-A7RE-11A-01-TS1.57401526-EF9E-49AC-8FF6-B4F9652311CE.svs
\\10.99.68.54\Digital pathology image lib\HubMap Skin TMC project\CellViT\inference_pilot\StarDist_2_23_24\json\TCGA-V5-A7RE-11A-01-TS1.57401526-EF9E-49AC-8FF6-B4F9652311C.json
centroid passed: [22, 10768]
centroid passed: [28, 10060]
centroid passed: [8, 10706]
centroid passed: [26, 10722]
centroid passed: [4, 9946]
centroid passed: [6, 10068]
centroid passed: [26, 10222]
centroid passed: [12, 10686]
centroid passed: [20, 10096]
centroid passed: [10, 10782]
centroid passed: [10, 10576]
centroid passed: [14, 10298]
centroid passed: [22, 10916]
centroid passed: [26, 10838]
centroid passed: [10, 11020]
centroid passed: [12, 11098]
centroid passed: [20, 10848]


In [24]:
mat_pth = os.path.join(pkl_pth,'nuclei_features_mats')
if not os.path.exists(mat_pth): os.mkdir(mat_pth)
print(mat_pth)

\\10.99.68.54\Digital pathology image lib\HubMap Skin TMC project\CellViT\inference_pilot\StarDist_2_23_24\json\nuclei_features_pkls\nuclei_features_mats


In [28]:
import pickle
from scipy.io import savemat

In [29]:
# I am pretty sure that this will work but I couldn't test it on the folder I was working with bc I don't have permission but just ask chatGPT if it doesn't work

dfs = [os.path.join(pkl_pth,f) for f in os.listdir(pkl_pth)]

for dfnm in dfs:
    
    outnm = os.path.join(mat_pth,os.path.basename(dfnm))
    
    print("Saving: {}".format(dfnm))
        
    with open(os.path.join(dfnm), 'rb') as f:
        df = pickle.load(f)

    col_names = df.columns.tolist()
    df = [_ for _ in df.to_numpy()]
    df = np.array(df)
    
    savemat(outnm, {'features':df, 'feature_names':col_names})

Saving: \\10.99.68.54\Digital pathology image lib\HubMap Skin TMC project\CellViT\inference_pilot\StarDist_2_23_24\json\nuclei_features_pkls\nuclei_features_mats


PermissionError: [Errno 13] Permission denied: '\\\\10.99.68.54\\Digital pathology image lib\\HubMap Skin TMC project\\CellViT\\inference_pilot\\StarDist_2_23_24\\json\\nuclei_features_pkls\\nuclei_features_mats'